# Modelling with Time Series

#####

##### The cell below extracts the history of the default rates per credit rating and is input into a Time Series model
##### This will allow the model to forecast the future expected default rates, under Mild, Base and Severe scenarios

In [115]:


import pandas as pd
import numpy as np
import statsmodels.api as sm
from pathlib import Path


wide_csv_path = Path("clean_credit_default_rates_wide.csv")  # adjust if needed


wide = pd.read_csv(wide_csv_path)
# normalize column names
wide.columns = [c.strip() for c in wide.columns]
year_col = [c for c in wide.columns if c.lower().startswith("year")][0]
wide = wide.set_index(year_col).sort_index()


ratings = [c for c in ["Aaa","Aa","A","Baa","Ba","B","Caa-C"] if c in wide.columns]

# put this helper once, above the loop
def horizon_sigma(phi, sigma, h):
    # σ_h = σ * sqrt( sum_{j=0}^{h-1} φ^(2j) )
    if abs(phi) < 0.999:
        return sigma * ((1 - (phi**2)**h) / (1 - phi**2))**0.5
    else:
        # safe fallback if phi≈1
        return sigma * (h**0.5)


# Helper: fit AR(1) with sensible fallback
def fit_ar1(series: pd.Series):
    series = series.dropna()
    info = {"n_obs": len(series)}
    if len(series) < 5:
        # Fallback: use mean level, zero persistence, sigma from series
        mu = float(series.mean())
        sigma = float(series.std(ddof=1)) if len(series) > 1 else 0.0
        q95 = float(np.nanpercentile(series.values, 95)) if len(series) > 1 else mu
        return dict(alpha=mu*(1-0.0), phi=0.0, sigma=sigma, last_dr=float(series.iloc[-1]), q95=q95, **info)

    y = series.iloc[1:].values
    x = sm.add_constant(series.iloc[:-1].values)
    model = sm.OLS(y, x, missing="drop").fit()
    alpha, phi = model.params
    sigma = float(model.resid.std(ddof=1))
    last_dr = float(series.iloc[-1])
    q95 = float(np.nanpercentile(series.values, 95))
    # Guardrails
    if abs(phi) > 0.98:  # avoid explosive forecasts
        mu = float(series.mean())
        alpha, phi = mu*(1-0.0), 0.0
    return dict(alpha=float(alpha), phi=float(phi), sigma=sigma, last_dr=last_dr, q95=q95, **info)

# Fit AR(1) per rating
params = []
for r in ratings:
    p = fit_ar1(wide[r])
    p["rating"] = r
    params.append(p)
ar_params = pd.DataFrame(params)[["rating","alpha","phi","sigma","last_dr","q95","n_obs"]]

# Forecast 5 or any year Base/Mild/Severe per rating
H = 10
rows = []
for _, row in ar_params.iterrows():
    base = [row["last_dr"]]
    for _ in range(H):
        nxt = row["alpha"] + row["phi"] * base[-1]
        base.append(max(nxt, 0.0))
    base = base[1:]

    # --- horizon-scaled bands (fan out with h) ---
    mild, severe = [], []
    for h in range(1, H+1):
        sig_h = horizon_sigma(row["phi"], row["sigma"], h)
        m = max(base[h-1] - sig_h, 0.0)                 # ~1σ below
        s = min(base[h-1] + 2.0*sig_h, row["q95"])      # ~2σ above, capped at q95
        mild.append(m)
        severe.append(s)

    for h in range(1, H+1):
        rows.append({
            "rating": row["rating"],
            "year_ahead": h,
            "DR_mild": mild[h-1],
            "DR_base": base[h-1],
            "DR_severe": severe[h-1],
        })

forecast = pd.DataFrame(rows)

# Quick views
print("AR(1) parameters per rating:")
print(ar_params.round(6).to_string(index=False))
print("\nYear 5 Mild/Base/Severe (by rating):")
print(forecast[forecast["year_ahead"]==5][["rating","DR_mild","DR_base","DR_severe"]].round(6).to_string(index=False))

# Optional: save for reuse
ar_params.to_csv("ar1_params_by_rating.csv", index=False)
forecast.to_csv("ar1_forecasts_5y.csv", index=False)  #name doesn't matter, data will be updated everytime this is ran



AR(1) parameters per rating:
rating    alpha       phi    sigma  last_dr      q95  n_obs
   Aaa 0.000000  0.000000 0.000000   0.0000 0.000000    104
    Aa 0.000586 -0.005968 0.001733   0.0000 0.004850    104
     A 0.000508  0.392944 0.002341   0.0015 0.003925    104
   Baa 0.001204  0.497571 0.003791   0.0011 0.010740    104
    Ba 0.005157  0.480023 0.014090   0.0073 0.037935    104
     B 0.016183  0.468694 0.032810   0.0072 0.101830    104
 Caa-C 0.070930  0.314649 0.104467   0.0916 0.333300    104

Year 5 Mild/Base/Severe (by rating):
rating  DR_mild  DR_base  DR_severe
   Aaa      0.0 0.000000   0.000000
    Aa      0.0 0.000582   0.004048
     A      0.0 0.000843   0.003925
   Baa      0.0 0.002357   0.010740
    Ba      0.0 0.009850   0.037935
     B      0.0 0.029932   0.101830
 Caa-C      0.0 0.103457   0.323571


##### Below are the dataframe of our porfolio

In [116]:
bonds1 = pd.DataFrame({

    "id": [
        "Verizon","Deutsche","Anglian Water (Osprey) Financing Plc","Citigroup Inc.","Intesa Sanpaolo S.p.A.",
        "Santander UK plc","Standard Chartered PLC","Vontobel Holding AG","CVS Health Corporation",
        "DENTSPLY SIRONA Inc.","Rogers Memorial Hospital Inc.","Baglan Moor Healthcare Plc",
        "Ochsner LSU Health System of North Louisiana","AT&T Inc.","Bell Canada, Inc.",
        "British Telecommunications plc","NBN Co Limited","Energy Transfer LP",
        "Keenan Fort Detrick Energy, LLC","PacifiCorp","TechnipFMC plc"
    ],

    "rating": [
        "Baa","A","Baa","Baa","Baa","Baa","Ba","Ba","Ba","Ba",
        "A","Aa","Baa","Baa","Ba","Baa","Aa","Ba","Aa","Baa","Baa"
    ],
    "security_class": [
        "Senior Secured","Senior Secured","Senior Secured","Senior Unsecured","Senior Unsecured",
        "Subordinate","Junior Subordinate","Subordinate","Junior Subordinate","Junior Subordinate",
        "Senior Secured","Secured","Senior Secured","Senior Unsecured","Junior Subordinate",
        "Senior Unsecured","Senior Unsecured","Junior Subordinate","Senior Secured","Junior Subordinate",
        "Senior Unsecured"
    ],
    "weight": [
        0.11,0.11,0.02,0.07,0.02,0.08,0.02,0.08,0.02,0.08,
        0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.08,0.02,0.02,0.03
    ]
})


In [117]:
import pandas as pd


ETF_COMPOSITION = {
    "IBHI": {"Baa": 0.0007, "Ba": 0.4398, "B": 0.3917, "Caa-C": 0.1570},
    "BNDX": {"Aa": 0.0187, "A": 0.0414, "Baa": 0.4070, "Ba": 0.5292},
    "PGHY": {"Aa": 0.2472, "A": 0.2662, "Baa": 0.2756, "Ba": 0.1843, "B": 0.0268}
}


ETF_WEIGHTS = {"IBHI": 0.02, "BNDX": 0.02, "PGHY": 0.06}


security_split = {"Senior Unsecured": 0.5, "Subordinate": 0.5}

# 
rows = []
for etf, comp in ETF_COMPOSITION.items():
    etf_weight = ETF_WEIGHTS[etf]
    for rating, pct in comp.items():
        for sec, sec_share in security_split.items():
            rows.append({
                "id": f"{etf}-{rating}-{sec}",
                "instrument": etf,
                "rating": rating,
                "security_class": sec,
                "weight": etf_weight * pct * sec_share
            })

etf_df = pd.DataFrame(rows)

#### Sanity checks below, running solely on the portfolio without the ETFs, just for model testing and validation

In [118]:


import pandas as pd

y1 = pd.read_csv("ar1_forecasts_5y.csv")
y1 = y1[y1["year_ahead"]==5].copy()   # change this accordingly == x



lgd_map = {    #stressed LGD assumptions
    "Senior Secured": 0.50,
    "Secured": 0.50,
    "Senior Unsecured": 0.65,
    "Senior Subordinate": 0.75,
    "Subordinate": 0.80,
    "Junior Subordinate": 0.95,
    "All Bonds": 0.75
}

bonds1["LGD"] = bonds1["security_class"].map(lgd_map)

def portfolio_edl(toy_df, y1_df, scenario_col):
    merged = toy_df.merge(y1_df[["rating", scenario_col]].rename(columns={scenario_col:"DR"}), on="rating", how="left")
    merged["EDL"] = merged["DR"] * merged["LGD"]              
    port_edl = (merged["EDL"] * merged["weight"]).sum()       
    return port_edl, merged

# Compute for each scenario
for sc_col, label in [("DR_base","Base"), ("DR_severe","Severe")]:
    port_loss, detail = portfolio_edl(bonds1, y1, sc_col)
    print(f"\nPortfolio 5-year EDL ({label}): {port_loss*100:.3f}%")
    print(detail[["id","rating","weight","LGD","DR","EDL"]].round(6).to_string(index=False))



Portfolio 5-year EDL (Base): 0.338%
                                          id rating  weight  LGD       DR      EDL
                                     Verizon    Baa    0.11 0.50 0.002357 0.001179
                                    Deutsche      A    0.11 0.50 0.000843 0.000421
        Anglian Water (Osprey) Financing Plc    Baa    0.02 0.50 0.002357 0.001179
                              Citigroup Inc.    Baa    0.07 0.65 0.002357 0.001532
                      Intesa Sanpaolo S.p.A.    Baa    0.02 0.65 0.002357 0.001532
                            Santander UK plc    Baa    0.08 0.80 0.002357 0.001886
                      Standard Chartered PLC     Ba    0.02 0.95 0.009850 0.009358
                         Vontobel Holding AG     Ba    0.08 0.80 0.009850 0.007880
                      CVS Health Corporation     Ba    0.02 0.95 0.009850 0.009358
                        DENTSPLY SIRONA Inc.     Ba    0.08 0.95 0.009850 0.009358
               Rogers Memorial Hospital Inc.      

In [ ]:
# Testing the parametric PD bounds calculation
import pandas as pd
import numpy as np

FCAST_PATH = "ar1_forecasts_5y.csv"         
PARAMS_PATH = "ar1_params_by_rating.csv"    


H = 5
SCENARIO_COL = "DR_base"   




f = pd.read_csv(FCAST_PATH)
drH_base = f.loc[f["year_ahead"]==H, ["rating", SCENARIO_COL]].rename(columns={SCENARIO_COL:"DR"})
m = bonds1.merge(drH_base, on="rating", how="left")
portfolio_PD_base = (m["weight"] * m["DR"]).sum()
portfolio_EDL_base = (m["weight"] * m["LGD"] * m["DR"]).sum()

print(f"[Single-number] Year+{H} portfolio PD (Base): {portfolio_PD_base*100:.3f}%")
print(f"[Single-number] Year+{H} portfolio EDL (Base): {portfolio_EDL_base*100:.3f}%")


Z95, Z99 = 1.96, 2.576
ar = pd.read_csv(PARAMS_PATH).set_index("rating")

def hstep_var(phi, sigma, h):
    # sigma = residual s.d. from AR(1) one-step; accumulate for h-steps ahead
    if abs(phi) == 1.0:
        return (sigma**2) * h
    return (sigma**2) * (1 - (phi**2)**h) / (1 - phi**2)


base_by_rating = drH_base.set_index("rating")["DR"].to_dict()


bounds = {}
for r, row in ar.iterrows():
    if r not in base_by_rating: 
        continue
    mu_h = float(base_by_rating[r])               # use your forecasted Base mean at H
    var_h = hstep_var(row["phi"], row["sigma"], H)
    sd_h  = float(np.sqrt(max(var_h, 0.0)))
    ub95  = mu_h + Z95 * sd_h
    ub99  = mu_h + Z99 * sd_h
    # clip to [0,1] and optionally cap at historical 95th percentile
    ub95 = max(0.0, min(ub95, 1.0, row["q95"]))
    ub99 = max(0.0, min(ub99, 1.0, row["q95"]))
    bounds[r] = dict(PD_base=mu_h, PD_ub95=ub95, PD_ub99=ub99)


PD95 = PD99 = 0.0
EDL95 = EDL99 = 0.0
for _, hld in bonds1.iterrows():
    r = hld["rating"]
    if r not in bounds: 
        continue
    PD95 += hld["weight"] * bounds[r]["PD_ub95"]
    PD99 += hld["weight"] * bounds[r]["PD_ub99"]
    EDL95 += hld["weight"] * hld["LGD"] * bounds[r]["PD_ub95"]
    EDL99 += hld["weight"] * hld["LGD"] * bounds[r]["PD_ub99"]

print(f"[Parametric] Year+{H} portfolio PD 95% ub: {PD95*100:.3f}% | 99% ub: {PD99*100:.3f}%")
print(f"[Parametric] Year+{H} portfolio EDL 95% : {EDL95*100:.3f}% | 99%   : {EDL99*100:.3f}%")


[Single-number] Year+5 portfolio PD (Base): 0.407%
[Single-number] Year+5 portfolio EDL (Base): 0.338%
[Parametric] Year+5 portfolio PD 95% ub: 1.653% | 99% ub: 1.659%
[Parametric] Year+5 portfolio EDL 95% : 1.356% | 99%   : 1.359%


###### Combining both ETFs and Bonds to construct entire portfolio for calculations (Personal workings)

In [94]:
import pandas as pd

# 1) Combine bonds + ETF slices
portfolio = pd.concat([bonds1, etf_df], ignore_index=True)

# 2) Choose horizon and scenario column
h = 5                          # e.g., 5-year forecast
scenario = "DR_base"           # or DR_mild / DR_severe

# 3) Pull the rating forecast for that horizon
dr_h = (forecast[forecast["year_ahead"] == h]
        .loc[:, ["rating", scenario]]
        .rename(columns={scenario: "DR"}))

# 4) Merge DR onto portfolio lines
port_h = portfolio.merge(dr_h, on="rating", how="left")

# 5) Map LGD (you can plug your recovery-based LGDs here)

lgd_map = {    #stressed LGD assumptions
    "Senior Secured": 0.50,
    "Secured": 0.50,
    "Senior Unsecured": 0.65,
    "Senior Subordinate": 0.75,
    "Subordinate": 0.80,
    "Junior Subordinate": 0.95,
    "All Bonds": 0.75
}

port_h["LGD"] = port_h["security_class"].map(lgd_map)

# 6) Compute portfolio PD and EDL for the horizon
port_h["EDL"] = port_h["weight"] * port_h["DR"] * port_h["LGD"]
port_pd  = (port_h["weight"] * port_h["DR"]).sum()
port_edl = port_h["EDL"].sum()

print(f"Year+{h} portfolio PD ({scenario}): {100*port_pd:.3f}%")
print(f"Year+{h} portfolio EDL ({scenario}): {100*port_edl:.3f}%")


Year+5 portfolio PD (DR_base): 0.505%
Year+5 portfolio EDL (DR_base): 0.410%


###### Personal workings for the cell below, not final

In [120]:
FCAST_PATH  = "ar1_forecasts_5y.csv"
PARAMS_PATH = "ar1_params_by_rating.csv"
H            = 10
SCENARIO_COL = "DR_severe" 

# ---- Load AR(1) forecast and params ----
f  = pd.read_csv(FCAST_PATH)
ar = pd.read_csv(PARAMS_PATH).set_index("rating")

# horizon DR by rating (chosen scenario)
drH = (f.loc[f["year_ahead"]==H, ["rating", SCENARIO_COL]]
         .rename(columns={SCENARIO_COL:"DR"}))

# Merge DR to each line
fullH = portfolio.merge(drH, on="rating", how="left")

lgd_map = {    #stressed LGD assumptions
    "Senior Secured": 0.50,
    "Secured": 0.50,
    "Senior Unsecured": 0.65,
    "Senior Subordinate": 0.75,
    "Subordinate": 0.80,
    "Junior Subordinate": 0.95,
    "All Bonds": 0.75
}

fullH["LGD"] = fullH["security_class"].map(lgd_map)

# Single-number PD/EDL
PD_base  = (fullH["weight"] * fullH["DR"]).sum()
EDL_base = (fullH["weight"] * fullH["DR"] * fullH["LGD"]).sum()
print(f"[Full] Year+{H} portfolio PD (Base): {PD_base*100:.3f}%")
print(f"[Full] Year+{H} portfolio EDL (Base): {EDL_base*100:.3f}%")

# ---- Parametric bounds using AR(1) variance build-up ----
Z95, Z99 = 1.96, 2.576

def hstep_var(phi, sigma, h):
    # AR(1) h-step variance of the *innovation* sum; sigma is 1-step resid s.d.
    if abs(phi) == 1.0:
        return (sigma**2) * h
    return (sigma**2) * (1 - (phi**2)**h) / (1 - phi**2)

# Prepare per-rating base mean
base_by_rating = drH.set_index("rating")["DR"].to_dict()

# Compute per-rating upper bounds
bounds = {}
for r, row in ar.iterrows():
    if r not in base_by_rating:
        continue
    mu_h = float(base_by_rating[r])
    var_h = hstep_var(row["phi"], row["sigma"], H)
    sd_h  = float(np.sqrt(max(var_h, 0.0)))
    ub95  = max(0.0, min(mu_h + Z95*sd_h, 1.0, row["q95"]))
    ub99  = max(0.0, min(mu_h + Z99*sd_h, 1.0, row["q95"]))
    bounds[r] = {"PD_base":mu_h, "PD_ub95":ub95, "PD_ub99":ub99}

# Aggregate to portfolio
PD95 = PD99 = 0.0
EDL95 = EDL99 = 0.0
for _, hld in fullH.iterrows():
    r = hld["rating"]
    if r not in bounds:
        continue
    PD95 += hld["weight"] * bounds[r]["PD_ub95"]
    PD99 += hld["weight"] * bounds[r]["PD_ub99"]
    EDL95 += hld["weight"] * hld["LGD"] * bounds[r]["PD_ub95"]
    EDL99 += hld["weight"] * hld["LGD"] * bounds[r]["PD_ub99"]

print(f"[Full Parametric] Year+{H} portfolio PD 95% ub: {PD95*100:.3f}% | 99% ub: {PD99*100:.3f}%")
print(f"[Full Parametric] Year+{H} portfolio EDL 95% : {EDL95*100:.3f}% | 99%   : {EDL99*100:.3f}%")

[Full] Year+10 portfolio PD (Base): 2.006%
[Full] Year+10 portfolio EDL (Base): 1.611%
[Full Parametric] Year+10 portfolio PD 95% ub: 2.015% | 99% ub: 2.015%
[Full Parametric] Year+10 portfolio EDL 95% : 1.617% | 99%   : 1.617%


### Run the cell below for different scenarios, as this is more complete and slightly more robust

In [ ]:

import pandas as pd
import numpy as np

FCAST_PATH  = "ar1_forecasts_5y.csv"     
PARAMS_PATH = "ar1_params_by_rating.csv" 

H            = 5            
SCENARIO_COL = "DR_severe"      


f  = pd.read_csv(FCAST_PATH)
ar = pd.read_csv(PARAMS_PATH).set_index("rating")

# sanity: make sure the forecast actually has this horizon & scenario
if SCENARIO_COL not in f.columns:
    raise ValueError(f"{SCENARIO_COL} not found in {FCAST_PATH} columns: {list(f.columns)}")
if H not in set(f["year_ahead"].unique()):
    raise ValueError(f"H={H} not found in 'year_ahead' of {FCAST_PATH}. "
                     f"Available: {sorted(f['year_ahead'].unique())}")


drH = (f.loc[f["year_ahead"] == H, ["rating", SCENARIO_COL]]
         .rename(columns={SCENARIO_COL: "DR"}))


fullH = portfolio.merge(drH, on="rating", how="left")


lgd_map = {
    "Senior Secured":      0.50,
    "Secured":             0.50,
    "Senior Unsecured":    0.65,
    "Senior Subordinate":  0.75,
    "Subordinate":         0.80,
    "Junior Subordinate":  0.95,
    "All Bonds":           0.75,
}
fullH["LGD"] = fullH["security_class"].map(lgd_map).fillna(0.65)  


PD_scn  = (fullH["weight"] * fullH["DR"]).sum()
EDL_scn = (fullH["weight"] * fullH["DR"] * fullH["LGD"]).sum()
print(f"[Full] Year+{H} portfolio PD ({SCENARIO_COL.split('_')[1].title()}): {PD_scn*100:.3f}%")
print(f"[Full] Year+{H} portfolio EDL ({SCENARIO_COL.split('_')[1].title()}): {EDL_scn*100:.3f}%")

Z95, Z99 = 1.96, 2.576

def hstep_sd(phi, sigma, h):
   
    phi = float(phi)
    sigma = float(sigma)
    if abs(phi) < 0.15:
        return sigma * np.sqrt(h)              
    
    var_h = (sigma**2) * (1 - (phi**2)**h) / (1 - phi**2)
    return float(np.sqrt(max(var_h, 0.0)))

# per-rating base mean at H
base_by_rating = drH.set_index("rating")["DR"].to_dict()

# compute per-rating UB95 / UB99
bounds = {}
for r, row in ar.iterrows():
    if r not in base_by_rating: 
        continue
    mu_h = float(base_by_rating[r])
    sd_h = hstep_sd(row["phi"], row["sigma"], H)
    ub95 = mu_h + Z95 * sd_h
    ub99 = mu_h + Z99 * sd_h
    
    q95  = float(row.get("q95", 1.0))
    ub95 = max(0.0, min(ub95, 1.0, q95))
    ub99 = max(0.0, min(ub99, 1.0, q95))
    bounds[r] = {"PD_base": mu_h, "PD_ub95": ub95, "PD_ub99": ub99}

# aggregate portfolio UB95 / UB99
PD95 = PD99 = 0.0
EDL95 = EDL99 = 0.0
for _, hld in fullH.iterrows():
    r = hld["rating"]
    if r not in bounds: 
        continue
    PD95  += hld["weight"] * bounds[r]["PD_ub95"]
    PD99  += hld["weight"] * bounds[r]["PD_ub99"]
    EDL95 += hld["weight"] * hld["LGD"] * bounds[r]["PD_ub95"]
    EDL99 += hld["weight"] * hld["LGD"] * bounds[r]["PD_ub99"]

print(f"[Full Parametric] Year+{H} portfolio PD 95% ub: {PD95*100:.3f}% | 99% ub: {PD99*100:.3f}%")
print(f"[Full Parametric] Year+{H} portfolio EDL 95% : {EDL95*100:.3f}% | 99%   : {EDL99*100:.3f}%")

# # (optional) quick debug view of per-rating DR at H
# dbg = fullH[["id","rating","weight","security_class","LGD","DR"]].sort_values("rating")
# # print(dbg.to_string(index=False))  # uncomment if you want to inspect


[Full] Year+5 portfolio PD (Severe): 2.006%
[Full] Year+5 portfolio EDL (Severe): 1.611%
[Full Parametric] Year+5 portfolio PD 95% ub: 2.015% | 99% ub: 2.015%
[Full Parametric] Year+5 portfolio EDL 95% : 1.617% | 99%   : 1.617%


### Finalized dataframe from running the models at different horizons and different severity scenarios

In [113]:
import pandas as pd

# --- Construct your summary table ---
data = [
    [5,  "Base",   0.00505, 0.00410, 0.02004, 0.02015, 0.01610, 0.01617],
    [7,  "Base",   0.00509, 0.00412, 0.02004, 0.02015, 0.01610, 0.01617],
    [10, "Base",   0.00510, 0.00413, 0.02004, 0.02015, 0.01610, 0.01617],
    [5,  "Severe", 0.02006, 0.01611, 0.02015, 0.02015, 0.01617, 0.01617],
    [7,  "Severe", 0.02006, 0.01611, 0.02015, 0.02015, 0.01617, 0.01617],
    [10, "Severe", 0.02006, 0.01611, 0.02015, 0.02015, 0.01617, 0.01617],
]

cols = [
    "Horizon (yrs)", "Scenario",
    "PD (Base)", "EDL (Base)",
    "PD 95% UB", "PD 99% UB",
    "EDL 95% UB", "EDL 99% UB"
]

results_table = pd.DataFrame(data, columns=cols)

# Format neatly as percentages with 3 decimals
formatted_table = results_table.copy()
for c in cols[2:]:
    formatted_table[c] = (results_table[c] * 100).round(3).astype(str) + "%"

print("Portfolio Stress Test Summary Table:")
print(formatted_table.to_string(index=False))

# # Optional: save as CSV for reporting
# formatted_table.to_csv("portfolio_stress_summary.csv", index=False)


Portfolio Stress Test Summary Table:
 Horizon (yrs) Scenario PD (Base) EDL (Base) PD 95% UB PD 99% UB EDL 95% UB EDL 99% UB
             5     Base    0.505%      0.41%    2.004%    2.015%      1.61%     1.617%
             7     Base    0.509%     0.412%    2.004%    2.015%      1.61%     1.617%
            10     Base     0.51%     0.413%    2.004%    2.015%      1.61%     1.617%
             5   Severe    2.006%     1.611%    2.015%    2.015%     1.617%     1.617%
             7   Severe    2.006%     1.611%    2.015%    2.015%     1.617%     1.617%
            10   Severe    2.006%     1.611%    2.015%    2.015%     1.617%     1.617%
